In [ ]:
from collections import deque
import math
import csv
import cv2 as cv2
import numpy as np
import mediapipe as mp
import itertools
from sklearn.metrics import accuracy_score
import warnings
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

warnings.filterwarnings("ignore")

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(model_complexity=0)
mp_drawing = mp.solutions.drawing_utils
W, H = 160*4, 160*3

In [ ]:
class CvFpsCalc(object):
    def __init__(self, buffer_len=10):
        self._start_tick = cv2.getTickCount()
        self._freq = 1000.0 / cv2.getTickFrequency()
        self._difftimes = deque(maxlen=buffer_len)

    def get(self):
        current_tick = cv2.getTickCount()
        different_time = (current_tick - self._start_tick) * self._freq
        self._start_tick = current_tick

        self._difftimes.append(different_time)

        fps = 1000.0 / (sum(self._difftimes) / len(self._difftimes))
        fps_rounded = round(fps, 2)

        return fps_rounded

In [ ]:
def logCsv(key, landmarklist):
    if key != -1:
        print(chr(key))
        with open("data.csv", mode="a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([key, *landmarklist])

In [ ]:
def shiftNormalized(frame, landmarkList):
    # Check if landmarkList is empty or all zeros
    if not np.any(landmarkList):
        return landmarkList
    # Calculate the minimum and maximum coordinates of the landmark points
    min_x = np.min([landmark[0] for landmark in landmarkList if landmark[0] > 0])
    min_y = np.min([landmark[1] for landmark in landmarkList if landmark[1] > 0])
    max_x = np.max([landmark[0] for landmark in landmarkList if landmark[0] > 0])
    max_y = np.max([landmark[1] for landmark in landmarkList if landmark[1] > 0])

    # Calculate the size of the hand
    hand_width = max_x - min_x
    hand_height = max_y - min_y

    # Shift and scale the landmark points
    for landmark in landmarkList:
        if landmark[0] > 0 and landmark[1] > 0:
            landmark[0] = (landmark[0] - min_x) / hand_width * W
            landmark[1] = (landmark[1] - min_y) / hand_height * H

    for landmark in landmarkList:
        x, y = landmark
        cv2.circle(frame, (int(x), int(y)), 5, (255, 255, 255), -1)

    return landmarkList

In [ ]:
def train():
    # Load the data into a pandas DataFrame
    df = pd.read_csv('data.csv')

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.3, random_state=42)

    rfc = RandomForestClassifier(n_estimators=50,
                                 max_depth=5,
                                 min_samples_split=2,
                                 min_samples_leaf=1,
                                 max_features='sqrt',
                                 bootstrap=True,
                                 random_state=42,
                                 n_jobs=-1)
    # Train the model
    rfc.fit(X_train, y_train)

    # Create the KNN classifier
    knn = KNeighborsClassifier(n_neighbors=10)

    # Train the model
    knn.fit(X_train, y_train)

    # Test the model
    knn_pred = knn.predict(X_test)
    rfc_pred = rfc.predict(X_test)

    # Calculate the accuracy of the model
    knn_acc = accuracy_score(y_test, knn_pred)
    rfc_acc = accuracy_score(y_test, rfc_pred)

    print("Knn_accuracy: {:.2f}%".format(knn_acc * 100))
    print("Rfc_accuracy: {:.2f}%".format(rfc_acc * 100))
    print()
    # Save the trained model to a file
    joblib.dump(knn, 'knn_model.joblib')
    joblib.dump(rfc, 'rfc_model.joblib')

In [ ]:
def calc_bounding_rect(landmarks):
    min_x, min_y, max_x, max_y = float('inf'), float('inf'), -float('inf'), -float('inf')
    for lm in landmarks.landmark:
        x, y = int(lm.x * W), int(lm.y * H)
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x)
        max_y = max(max_y, y)
    return min_x, min_y, max_x - min_x, max_y - min_y


In [ ]:
def main():
    train()
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H)

    cvFpsCalc = CvFpsCalc()
    save_mode = False
    # rfc know what it doesn't know
    # knn sure even if it doesn't now
    clf = joblib.load('rfc_model.joblib')

    while True:
        key = cv2.waitKey(1)

        if key == 27:
            break

        totalHand = np.zeros((H, W, 3), dtype=np.uint8)

        ret, frame = cap.read()
        if not ret:
            raise Exception("Failed at capturing frame")

        frame = cv2.flip(frame, 1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame.flags.writeable = False
        results = hands.process(frame)
        frame.flags.writeable = True

        fps = cvFpsCalc.get()

        landmarkList = [[0] * 2 for _ in range(42)]

        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(totalHand, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                if handedness.classification[0].label == 'Left':
                    # Update first 21 elements for left hand landmarks
                    for i, lm in enumerate(hand_landmarks.landmark):
                        landmarkList[i][0] = lm.x * W
                        landmarkList[i][1] = lm.y * H
                if handedness.classification[0].label == 'Right':
                    # Update last 21 elements for right hand landmarks
                    for i, lm in enumerate(hand_landmarks.landmark):
                        landmarkList[i+21][0] = lm.x * W
                        landmarkList[i+21][1] = lm.y * H

        #may use sklearn min max scaler
        landmarkList = shiftNormalized(totalHand, landmarkList)
        cv2.rectangle(frame, (0,0), (200,40), (245,100,50), -1)
        cv2.putText(frame, f"FPS: {fps}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                   1.0, (255, 255, 255), 4, cv2.LINE_AA)

        oneDimLandmarks = list(itertools.chain(*landmarkList))

        if save_mode:
            logCsv(key, oneDimLandmarks)

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if key == 13:
            save_mode = True

        prediction = clf.predict_proba(np.array(oneDimLandmarks).reshape(1, -1))[0]
        max_prob_idx = np.argmax(prediction)
        max_prob = prediction[max_prob_idx]

        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                if handedness.classification[0].label == 'Right':
                    hand_landmarks = results.multi_hand_landmarks[0]
                    brect = calc_bounding_rect(hand_landmarks)

                    # Add the predicted letter and probability to the bounding box
                    text = f"{chr(max_prob_idx + 97)} (%{max_prob*100:.2f})"
                    cv2.putText(frame, text, (brect[0] + 10, brect[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


        cv2.imshow("totalHands", totalHand)
        cv2.imshow('Hand State Detection', frame)

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
import cProfile
import pstats

if __name__ == '__main__':
    cProfile.run('main()', 'stats')
    p = pstats.Stats('stats')
    p.sort_stats('cumtime').print_stats(5)


Right   : A B C D E

Left    :


In [ ]:
df = pd.read_csv('data.csv', header=None)
l = list(df.groupby(0)[1].count().index)
print(l)
for i in l:
    print(chr(i), (df[0] == i).sum())